In [ ]:
# Disable widget-based progress bars
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Use simple progress bars instead of widgets
import transformers
transformers.logging.set_verbosity_error()


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Point to local model directory
model_id = "./gpt-oss-120b/"  # Changed from "openai/gpt-oss-120b"

# Load model and tokenizer
print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # or "auto"
    device_map="auto",
    trust_remote_code=True,
    local_files_only=True  # Optional: ensures it doesn't try to download from Hub
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    local_files_only=True  # Optional: ensures it doesn't try to download from Hub
)

# Test inference
messages = [
    {"role": "user", "content": "What is 2+2?"}
]

# Apply chat template (harmony format)
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

inputs = tokenizer(text, return_tensors="pt").to(model.device)

# Generate
print("Generating...")
outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=False
)

response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
print("\nResponse:", response)

⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 37 files:   0%|                                 | 0/37 [00:00<?, ?it/s]Downloading '.gitattributes' to 'gpt-oss-120b/.cache/huggingface/download/wPaCkH-WbT7GsmxMKKrNZTV4nSM=.52373fe24473b1aa44333d318f578ae6bf04b49b.incomplete'

.gitattributes: 1.57kB [00:00, 11.0MB/s]
Download complete. Moving file to gpt-oss-120b/.gitattributes
Fetching 37 files:   3%|▋                        | 1/37 [00:00<00:12,  2.94it/s]Downloading 'config.json' to 'gpt-oss-120b/.cache/huggingface/download/8_PA_wEVGiVa2goH2H4KQOQpvVY=.42300b8993060d365b1d138a35998eb70c745285.incomplete'

chat_template.jinja: 16.7kB [00:00, 7.27MB/s]
Download complete. Moving file to gpt-oss-120b/chat_template.jinja

config.json: 0.00B [00:00, ?B/s]Downloading 'generation_config.json' to 'gpt-oss-120b/.cache/huggingface/download/3EVKVggOldJcKSsGjSdoUCN1AyQ=.86f91466555bd40e3de0b1edee3d5d82f4ccdbfe.incomplete'
config.json: 2.09kB [00:00, 5.24MB